In [ ]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import numpy as np
# !pip3 install transformers
# !pip3 install simpletransformers
# # %%writefile setup.sh
# !git clone https://github.com/NVIDIA/apex
# !pip3 install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

In [ ]:
from google.colab import drive
drive.mount('/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive/


In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

file = '/gdrive/My Drive/NLA_project/train.csv'
# data = pd.read_csv(file, names=["Ref_paper", "Cit_paper", "text_a", "text_b", "labels"])
data = pd.read_csv(file)
data.head()


,Ref_name,Cit_name,Cit_Text,Ref_text,label
0,C02-1025,C10-2104,case neither global feature chieu ng aggregate...,global feature extracted occurrence token whol...,1
1,C02-1025,C10-2104,case neither global feature chieu ng aggregate...,muc best result achieved sra krupka,0
2,C02-1025,C10-2104,case neither global feature chieu ng aggregate...,similarly token tested list found correspondin...,0
3,C02-1025,C10-2104,case neither global feature chieu ng aggregate...,need initcaps considered feature,0
4,C02-1025,C10-2104,case neither global feature chieu ng aggregate...,muc muc named entity task defined finding foll...,0


In [ ]:
data.rename(columns = {'Ref_text':'text_b', 'Cit_Text': 'text_a', 'label':'labels'}, inplace = True)
data['text_a'] = data['text_a'].astype(str)
data['text_b'] = data['text_b'].astype(str)
# data["labels"] = data["labels"].apply(lambda x: list(x))

data.head()

,Ref_name,Cit_name,text_a,text_b,labels
0,C02-1025,C10-2104,case neither global feature chieu ng aggregate...,global feature extracted occurrence token whol...,1
1,C02-1025,C10-2104,case neither global feature chieu ng aggregate...,muc best result achieved sra krupka,0
2,C02-1025,C10-2104,case neither global feature chieu ng aggregate...,similarly token tested list found correspondin...,0
3,C02-1025,C10-2104,case neither global feature chieu ng aggregate...,need initcaps considered feature,0
4,C02-1025,C10-2104,case neither global feature chieu ng aggregate...,muc muc named entity task defined finding foll...,0


In [ ]:
def get_f1(y_true, y_pred):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for a,b in zip(y_true, y_pred):
      if a==1 and b==1:
        tp+=1
      elif a==1 and b==0:
        fn+=1
      elif a==0 and b==1:
        fp+=1
      elif a==0 and b==0:
        tn+=1
    precision = tp / tp + fp + 1
    recall = tp / tp + fn + 1
    return 2*(precision*recall)/(precision+recall+0.001)

def recall(y_true, y_pred):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for a,b in zip(y_true, y_pred):
      if a==1 and b==1:
        tp+=1
      elif a==1 and b==0:
        fn+=1
      elif a==0 and b==1:
        fp+=1
      elif a==0 and b==0:
        tn+=1
    precision = tp / tp + fp + 1
    recall = tp / tp + fn + 1
    return recall

def precision(y_true, y_pred):
    tp = 0
    fp = 0
    tn = 0
    fn = 0
    for a,b in zip(y_true, y_pred):
      if a==1 and b==1:
        tp+=1
      elif a==1 and b==0:
        fn+=1
      elif a==0 and b==1:
        fp+=1
      elif a==0 and b==0:
        tn+=1
    precision = tp / tp + fp + 1
    recall = tp / tp + fn + 1
    return precision  

In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip3 install gputil
!pip3 install psutil
!pip3 install humanize

import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
    process = psutil.Process(os.getpid())
    print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |     Proc size: " + humanize.naturalsize(process.memory_info().rss))
    print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total     {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=730aaa828b56b93db82d2718ccb379cd22c360d69a2c87e67cfeec448a731e8c
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 26.1 GB  |     Proc size: 617.6 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total     11441MB


In [ ]:
train_args = {
    'reprocess_input_data': True,
    'overwrite_output_dir': True,
    'evaluate_during_training': True,
    'max_seq_length': 512,
    'num_train_epochs': 1,
    'evaluate_during_training_steps': 500,
    'wandb_project': 'sts-b-medium',
    'train_batch_size': 16,
    'output_dir': '/gdrive/My Drive/NLA_project/outputs/',
   'cache_dir': '/gdrive/My Drive/NLA_project/cache/'
}
data['text_a'] = data['text_a'].astype(str)
data['text_b'] = data['text_b'].astype(str)
# data = data.applymap(str)
msk = np.random.rand(len(data)) < 0.8
train_df = data[msk]
eval_df = data[~msk]
model = ClassificationModel('roberta', 'roberta-base', num_labels=2, args=train_args)
model.train_model(train_df, eval_df=eval_df)
print("Trained")
result, model_outputs, wrong_predictions = model.eval_model(eval_df, get_f1=get_f1,recall=recall,precision=precision)
print(result, model_outputs)

/usr/local/lib/python3.6/dist-packages/simpletransformers/classification/classification_model.py:179: UserWarning: wandb_project specified but wandb is not available. Wandb disabled.
  warnings.warn("wandb_project specified but wandb is not available. Wandb disabled.")


RuntimeError: ignored